<a href="https://colab.research.google.com/github/Nicsdiary/test-repo/blob/main/Keras_Practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
#데이터를 가져오는 부분

from keras.datasets import boston_housing
(train_data,train_labels),(test_data,test_labels)=boston_housing.load_data()

#Keras 라이브러리에서 Boston Housing 데이터셋을 불러와서, 학습 데이터와 레이블, 테스트 데이터와 레이블을 각각 train_data, train_labels, test_data, test_labels 변수에 저장하는 코드

In [105]:
print(train_data.shape)
print(test_data.shape)

#train_data와 test_data의 크기(shape)를 출력하는 코드입니다. 
# train_data는 404개의 샘플과 13개의 feature로 이루어진 2D 배열이며,
# test_data는 102개의 샘플과 13개의 feature로 이루어진 2D 배열입니다. 
#각각의 feature는 보스턴 지역의 주택 가격 예측 모델을 위한 입력값

(404, 13)
(102, 13)


In [106]:
print(train_data[0])

#첫 번째 훈련 데이터의 특성 값들을 출력하는 코드

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [107]:


mean = train_data.mean(axis=0)
train_data -= mean   #train_data = train_data - mean 과 같은 표현 
std = train_data.std(axis=0)


#데이터 전처리를 위한 Scaling 작업을 수행하는 코드

#train_data.mean(axis=0) : train_data의 열(axis=0) 별 평균값을 계산하여 변수 mean에 저장합니다.
#train_data -= mean : train_data에서 평균값(mean)을 빼서 중심을 0으로 만듭니다. 
#이는 각 특성(feature)의 중앙이 0이 되도록 만들어 줍니다. 이때, -= 연산자를 사용하여 train_data를 업데이트 합니다.
#train_data.std(axis=0) : train_data의 열(axis=0) 별 표준편차를 계산하여 변수 std에 저장합니다. 
#이는 각 특성(feature)의 분포가 비슷하도록 만들어 줍니다.

In [108]:
# 모델의 구성 
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1])))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',
                loss='mse',
                metrics=['mse'])
  return model


  # models.Sequential()로 모델 객체를 생성하고, model.add()로 층(layer)을 추가해나가는 방식으로 모델을 구성
  # 이 모델은 3개의 층으로 이루어져 있습니다. 
  # 첫 번째 층은 입력 데이터의 크기를 정의하기 위한 input_shape 파라미터와 64개의 유닛을 가진 fully-connected 층(layers.Dense(64))으로 구성되어있음, 활성화 함수로 ReLU를 사용합니다. 
  # 두 번째 층은 64개의 유닛을 가진 fully-connected 층(layers.Dense(64))으로 구성되어 있으며, 활성화 함수로 ReLU를 사용합니다. 
  # 마지막 층은 하나의 출력 유닛만을 가진 fully-connected 층(layers.Dense(1))으로 구성되어 있습니다. 
  # 이 모델은 optimizer='rmsprop'로 RMSprop 최적화 알고리즘을 사용하며, 손실 함수(loss function)로는 MSE(mean squared error)를 사용합니다
  # 모델의 성능 평가를 위해 MSE를 모니터링합니다. 마지막으로, 구성된 모델 객체를 반환

In [109]:
# K-folder 검증을 사용한 훈련 검증 

import numpy as np 

k = 4                                       # folder 개수
num_val_samples = len(train_data) // k      # 한 folder의 데이터 수 
all_scores = []


for i in range(k):
  print('처리중인 폴드 #', i)


  #검증 데이터의 준비 
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]

  # K-folder 검증(k-fold cross validation)을 사용하여 훈련 검증하는 방법을 구현
  # K-folder 검증은 모델을 검증하기 위해 데이터를 K개의 folder로 나누고, 각 folder를 순차적으로 검증에 사용하고 나머지 folder로 모델을 훈련하는 방식입니다. 
  # 이를 K번 반복하여 모든 folder가 검증에 사용

  # 위 코드에서는 K를 4로 설정하고, train_data의 총 데이터 수를 K로 나누어서 한 folder의 데이터 수를 계산합니다. 
  # for 루프를 이용하여 K개의 folder를 순차적으로 선택하여 검증 데이터(val_data)와 검증 레이블(val_labels)을 설정
  # 이 데이터와 레이블을 이용하여 모델을 검증하고 결과를 all_scores 리스트에 추가

  #학습 데이터의 준비
  data1 = train_data[:i * num_val_samples]
  data2 = train_data[(i+1) * num_val_samples :]
  data1_labels = train_labels[:i * num_val_samples]
  data2_labels = train_labels[(i+1) * num_val_samples :]

  partial_train_data = np.concatenate([data1, data2], axis=0)
  partial_train_labels = np.concatenate([data1_labels, data2_labels], axis=0)

  # data1은 train_data에서 0부터 i번째 폴드의 시작 인덱스 전까지의 데이터를 의미
  # data2는 i번째 폴드의 시작 인덱스부터 i+1번째 폴드의 시작 인덱스 전까지의 데이터를 의미
  # print(0, i * num_val_samples)와 print(i * num_val_samples, (i+1) * num_val_samples)
  # => 각 폴드의 데이터가 어디서부터 어디까지인지를 확인하기 위한 디버깅용 출력
  # partial_train_data는 data1과 data2를 axis=0 방향으로 합친 것으로, i번째 폴드를 제외한 나머지 폴드의 데이터를 훈련 데이터로 사용


  #폴드(fold)란 기계 학습에서 교차 검증(cross-validation)을 수행할 때, 전체 데이터를 일정한 크기의 '폴드' 혹은 '파티션'으로 나누어서 검증을 수행하는 단위를 말합니다.
  # 즉, 전체 데이터를 k개의 폴드로 나누어서 각각의 폴드를 검증 데이터(validation data)로 사용하고, 나머지 폴드를 훈련 데이터(training data)로 사용하는 것입니다. 
  #이 때, k개의 폴드로 나누는 것을 k-fold 교차 검증

 # 모델의 학습
  model = build_model()
  model.summary()

  model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0)

  val_mse, val_mae = model.evaluate(val_data, val_labels)
  print(val_mse, val_mae)
  all_scores.append(val_mae)


처리중인 폴드 # 0


TypeError: ignored